In [ ]:
"""
Equitable Food Distribution for NYC
-----------------------------------

Python implementation of the two-step optimization proposal:

Step 1: Minimizing supply gaps across neighborhoods and pantries
Step 2: Transportation / min-cost flow between pantries

Library: gurobipy
"""

import gurobipy as gp
from gurobipy import GRB


def build_step1_model(neighborhoods, pantries, D, c, C, K, E_np, R):
    """
    Step 1: Minimizing Supply Gap

    Sets
    ----
    neighborhoods : iterable
        Set of neighborhood indices/IDs N.
    pantries : iterable
        Set of pantry indices/IDs P.

    Parameters
    ----------
    D : dict
        Demand per neighborhood n, D[n].
    c : dict
        Cost of supplying pantry p, c[p].
    C : float or dict
        Capacity of pantries. If scalar, same C for all p.
        If dict, C[p] is capacity for pantry p.
    K : int
        Maximum number of (neighborhood, pantry) supply links.
        (In LaTeX: sum_n sum_p y_np <= K)
    E_np : dict
        Distance from neighborhood n to pantry p, E_np[(n, p)].
    R : float
        Maximum distance any pantry p can be from neighborhood n.

    Returns
    -------
    model : gp.Model
    s : gp.tupledict
        Continuous shipped quantities s[n, p] >= 0.
    y : gp.tupledict
        Binary assignment y[n, p] in {0, 1}.
    unmet : gp.tupledict
        Unmet demand per neighborhood (for clarity), u[n] >= 0.
    """

    m = gp.Model("step1_min_supply_gap")

    # --- Decision variables ---
    # s_np: amount of food supplied at each pantry p in each neighborhood n
    s = m.addVars(neighborhoods, pantries, lb=0.0, vtype=GRB.CONTINUOUS, name="s")

    # y_np: 1 if pantry p supplies neighborhood n, 0 otherwise
    y = m.addVars(neighborhoods, pantries, vtype=GRB.BINARY, name="y")

    # (Not in the LaTeX directly, but implied by "supply gap"):
    # unmet_n : unmet demand in each neighborhood, >= 0
    unmet = m.addVars(neighborhoods, lb=0.0, vtype=GRB.CONTINUOUS, name="unmet")

    # --- Objective function ---
    # LaTeX: min (D_n - sum_p s_np) + sum_{n,p} c_p y_np
    # That expression is ambiguous in indexing, so here we interpret:
    #   minimize ∑_n unmet_n + ∑_{n,p} c_p y_np
    # with unmet_n >= D_n - ∑_p s_np
    m.setObjective(
        gp.quicksum(unmet[n] for n in neighborhoods)
        + gp.quicksum(c[p] * y[n, p] for n in neighborhoods for p in pantries),
        GRB.MINIMIZE,
    )

    # --- Constraints ---

    # 1) Limit on total number of supply links:
    # ∑_n ∑_p y_np ≤ K
    m.addConstr(
        gp.quicksum(y[n, p] for n in neighborhoods for p in pantries) <= K,
        name="max_links",
    )

    # 2) Capacity constraints:
    # ∑_n s_np ≤ C * y_np   ∀ p
    # If C is scalar, same for all p. If dict, use C[p].
    for p in pantries:
        if isinstance(C, dict):
            cap_p = C[p]
        else:
            cap_p = C
        m.addConstr(
            gp.quicksum(s[n, p] for n in neighborhoods) <= cap_p * y.sum("*", p),
            name=f"capacity_p_{p}",
        )

    # 3) Demand coverage:
    # ∑_p s_np ≥ D_n   ∀ n
    for n in neighborhoods:
        m.addConstr(
            gp.quicksum(s[n, p] for p in pantries) + unmet[n] >= D[n],
            name=f"demand_n_{n}",
        )

    # 4) Distance constraints:
    # E_np * y_np ≤ R   ∀ n, ∀ p
    # NOTE: As you wrote in the LaTeX, this doesn't involve s_np yet.
    # You already flagged that this likely needs to involve shipped quantity;
    # here we just translate literally, with a TODO.
    for n in neighborhoods:
        for p in pantries:
            m.addConstr(
                E_np[(n, p)] * y[n, p] <= R,
                name=f"distance_n_{n}_p_{p}",
            )

    # 5) Link s_np and y_np (not in LaTeX but standard “big-M” style logic):
    # If y_np = 0, then s_np must be 0 (no shipments without a link).
    # This uses the same capacity bound as the pantry p.
    for n in neighborhoods:
        for p in pantries:
            if isinstance(C, dict):
                cap_p = C[p]
            else:
                cap_p = C
            m.addConstr(
                s[n, p] <= cap_p * y[n, p],
                name=f"link_s_y_n_{n}_p_{p}",
            )

    m.update()
    return m, s, y, unmet


def build_step2_model(pantries, neighborhoods, U, S, C_p, I_n, E_pp, R):
    """
    Step 2: Transportation / Min-Cost Flow between pantries

    Sets
    ----
    pantries : iterable
        Set of pantry indices/IDs P.
    neighborhoods : iterable
        Set of neighborhood indices/IDs N.

    Parameters
    ----------
    U : dict
        Unmet demand per neighborhood from Stage 1, U[n].
    S : dict
        Amount supplied to pantry p from Stage 1, S[p].
    C_p : dict
        Capacity of each pantry p, C_p[p].
    I_n : dict
        For each neighborhood n, list/set of pantries belonging to that neighborhood:
        I_n[n] = list of pantries.
    E_pp : dict
        Distance between pantries, E_pp[(p, p')].
    R : float
        Maximum radius (distance) between pantries for a feasible shipment.

    This function internally builds feasible sets F_p (pantries p' reachable from p).

    Returns
    -------
    model : gp.Model
    x : gp.tupledict
        Flow variables x[p, p'] >= 0.
    b : gp.tupledict
        Net change variables b[p] (can be positive or negative).
    u : gp.tupledict
        Unmet demand u[n] after reallocation.
    """

    m = gp.Model("step2_transport_min_cost_flow")

    # --- Build feasible sets F_p = {p' != p | E_pp' <= R or (p,p') in same neighborhood} ---
    pantries_list = list(pantries)

    # Precompute neighborhood membership for faster lookup:
    pantry_to_neighborhoods = {p: set() for p in pantries_list}
    for n in neighborhoods:
        for p in I_n.get(n, []):
            pantry_to_neighborhoods[p].add(n)

    F = {p: [] for p in pantries_list}
    for p in pantries_list:
        for p2 in pantries_list:
            if p2 == p:
                continue
            # Distance-based eligibility:
            close_enough = E_pp.get((p, p2), float("inf")) <= R

            # Or in same neighborhood (there exists n such that both are in I_n)
            same_neighborhood = len(
                pantry_to_neighborhoods[p].intersection(pantry_to_neighborhoods[p2])
            ) > 0

            if close_enough or same_neighborhood:
                F[p].append(p2)

    # --- Decision variables ---

    # x_pp': flow from pantry p to pantry p' (only if p' in F[p])
    x = m.addVars(
        [(p, p2) for p in pantries_list for p2 in F[p]],
        lb=0.0,
        vtype=GRB.CONTINUOUS,
        name="x",
    )

    # b_p: net change to pantry p's supply (can be positive or negative)
    # S_p' = S_p - b_p in the LaTeX text
    b = m.addVars(pantries_list, vtype=GRB.CONTINUOUS, name="b")

    # u_n: final unmet demand for neighborhood n
    u = m.addVars(neighborhoods, lb=0.0, vtype=GRB.CONTINUOUS, name="u")

    # --- Objective function ---
    # LaTeX: min ∑_{p, p'} x_pp' + ∑_n u_n
    m.setObjective(
        gp.quicksum(x[p, p2] for p in pantries_list for p2 in F[p])
        + gp.quicksum(u[n] for n in neighborhoods),
        GRB.MINIMIZE,
    )

    # --- Constraints ---

    # 1) Flow balance at each pantry:
    # ∑_{p' in F_p'} x_{p'p} - ∑_{p' in F_p} x_{pp'} = b_p   ∀ p
    # We need "incoming F_p'" = {p' | p in F[p']}.
    F_in = {p: [] for p in pantries_list}
    for p in pantries_list:
        for p2 in F[p]:
            F_in[p2].append(p)

    for p in pantries_list:
        incoming = gp.quicksum(x[p_in, p] for p_in in F_in[p]) if F_in[p] else 0.0
        outgoing = gp.quicksum(x[p, p_out] for p_out in F[p]) if F[p] else 0.0
        m.addConstr(
            incoming - outgoing == b[p],
            name=f"flow_balance_p_{p}",
        )

    # 2) No net creation of supply:
    # From S_p' = S_p - b_p, wanting total supply preserved gives:
    # ∑_p S_p = ∑_p S_p' = ∑_p (S_p - b_p)  =>  ∑_p b_p = 0
    m.addConstr(
        gp.quicksum(b[p] for p in pantries_list) == 0.0,
        name="no_net_supply_creation",
    )

    # 3) Capacity constraints:
    # b_p + S_p ≤ C_p   ∀ p
    for p in pantries_list:
        m.addConstr(
            b[p] + S[p] <= C_p[p],
            name=f"capacity_p_{p}",
        )

    # 4) Neighborhood unmet demand:
    # u_n = U_n - ∑_{p in I_n} b_p
    for n in neighborhoods:
        m.addConstr(
            u[n]
            == U[n]
            - gp.quicksum(b[p] for p in I_n.get(n, [])),
            name=f"unmet_n_{n}",
        )

    # Nonnegativity is already enforced via variable lb=0.0 for u[n] and x[p,p'].

    m.update()
    return m, x, b, u


if __name__ == "__main__":
    # ------------------------------------------------------------------
    # Example skeleton showing how to plug data in and solve.
    # Replace with your actual NYC data.
    # ------------------------------------------------------------------

    # --- Example sets ---
    neighborhoods = ["N1", "N2", "N3"]
    pantries = ["P1", "P2", "P3"]

    # --- Example parameters for Step 1 ---
    D = {"N1": 1000.0, "N2": 800.0, "N3": 600.0}   # demand per neighborhood
    c = {"P1": 1.0, "P2": 1.2, "P3": 0.9}         # cost per pantry
    C = 2000.0                                    # shared capacity or use dict per pantry
    K = 5                                         # max number of links
    R1 = 3.0                                      # max distance n-p

    # Distance neighborhood->pantry (dummy example)
    E_np = {}
    for n in neighborhoods:
        for p in pantries:
            # TODO: fill with real Euclidean distances
            E_np[(n, p)] = 1.0

    # Build and solve Step 1
    model1, s, y,


Set parameter Username
Set parameter LicenseID to value 2703836
Academic license - for non-commercial use only - expires 2026-09-05
Gurobi Optimizer version 12.0.3 build v12.0.3rc0 (mac64[arm] - Darwin 24.6.0 24G90)

CPU model: Apple M1 Pro
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 32 rows, 24 columns and 72 nonzeros
Model fingerprint: 0x5c7aad7a
Variable types: 12 continuous, 12 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [1e+00, 6e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [4e+00, 2e+02]
Found heuristic solution: objective 17.0000000
Presolve removed 17 rows and 7 columns
Presolve time: 0.00s
Presolved: 15 rows, 17 columns, 44 nonzeros
Variable types: 9 continuous, 8 integer (8 binary)

Root relaxation: objective -1.335000e+02, 9 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl 